In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/chenghin/anaconda3/envs/explainable-mutation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from model import Seq2Seq
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

pretrained_model_name = "microsoft/unixcoder-base"

# build model
tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_name)
config = RobertaConfig.from_pretrained(pretrained_model_name)
# import！！！you must set is_decoder as True for generation
config.is_decoder = True
encoder = RobertaModel.from_pretrained(pretrained_model_name,config=config) 

model = Seq2Seq(encoder=encoder,decoder=encoder,config=config,
                beam_size=10,max_length=256,
                sos_id=tokenizer.convert_tokens_to_ids(["<mask0>"])[0],eos_id=tokenizer.sep_token_id)

Updated!!!


In [4]:
import os
def load_model(model, load_model_path='fine_tuned_models'):
    model_to_load = model.module if hasattr(model, 'module') else model
    load_model_path = os.path.join(load_model_path, 'pytorch_model.bin')
    model.load_state_dict(torch.load(load_model_path, map_location='cpu'))
    model.to(device)
load_model(model, '/home/chenghin/Desktop/repos/java-mutation-framework/models/code-generation/saved_models/checkpoint-best-score')

In [5]:
def predict(model, code, gold):
    input_ids = tokenizer(code, return_tensors="pt").input_ids
    generated_ids = model.generate(input_ids.to(device))
    comment = tokenizer.decode(generated_ids[0][0], skip_special_tokens=True)
    return comment, getSmoothBLEU4(gold, comment)

In [6]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction
chencherry = SmoothingFunction()
def getSmoothBLEU4(gold, result):
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([gold.split()], result.split(), weights = [0.25,0.25,0.25,0.25], smoothing_function=chencherry.method2)
    return round(BLEUscore,4)

In [7]:
code = '''
public RequestMethodsRequestCondition getMatchingCondition(ServerWebExchange exchange) {
                if (CorsUtils.isPreFlightRequest(exchange.getRequest())) {
                        return matchPreFlight(exchange.getRequest());
                }
                if (getMethods().isEmpty()) {
                        if (RequestMethod.OPTIONS.name().equals(exchange.getRequest().getMethodValue())) {
                                return null; // We handle OPTIONS transparently, so don't match if no explicit declarations
                        }
                        return this;
                }
                return matchRequestMethod(exchange.getRequest().getMethod());
        }

'''

In [8]:
gold = 'check if any of the http request methods match the given request and return an instance that contains the matching http request method only'

In [9]:
predict(model, code, gold)

("public RequestMethodsRequestCondition getMatchingCondition(ServerWebExchange exchange) {\n                if (CorsUtils.isPreFlightRequest(exchange.getRequest())) {\n                        return matchPreFlight(exchange.getRequest());\n                }\n                if (getMethods().isEmpty()) {\n                        if (RequestMethod.OPTIONS.name().equals(exchange.getRequest().getMethodValue())) {\n                                return null; // We handle OPTIONS transparently, so don't match if no explicit declarations\n                        }\n                        return this;\n                }",
 0.0384)